# PROGRAMA DE VIGA PROTENDIDA

Algorítmo para cálculo de uma viga protendida. \n
Todas as grandezas apresentadas possuem como unidades base kN [força] e cm [comprimento].

In [46]:
import handcalcs.render
import forallpeople as si

# TODAS AS UNIDADES EM kN e cm

## 1 - Informações Iniciais:

### 1.1 - Geometria da Viga:

In [2]:
# Geometria da Viga:

Area = 2850
Inercia = 3.96 * 10 ** 6
W_i = 61500
W_s = 61500
e_p = 52
Vao = 2458

### 1.2 - Propriedades dos Materiais:

In [44]:
%%render
# Propriedades dos Materiais:

# Concreto:
f_ck = 4
f_ptk = 190
f_yk = 50
f_cj = 3.2

E_s = 21000
E_p = 20000

<IPython.core.display.Latex object>

In [4]:
%%render 2

E_ci = 5600*(f_ck*10)**(1/2)
E_cs = E_ci*(0.8+0.2*f_ck/8)

E_cit0 = (f_cj/f_ck)**(1/2)*E_ci
E_cst0 = E_cit0*(0.8+0.2*f_ck/8)

f_yd = f_yk/1.15
sigma_p0 = 0.8*f_ptk

<IPython.core.display.Latex object>

### 1.3 - Carregamentos:

In [5]:
%%render 2

#PESO PRÓPRIO DA VIGA:
Q_pp = 0.000025*Area
Q_sc = 0.088

<IPython.core.display.Latex object>

### 1.4 - Esforços no Centro do Vão:

In [6]:
%%render 2 short
M_F = (Q_pp+Q_sc)*Vao**2*0.125
M_0 = (Q_pp)*Vao**2*0.125
V_F = (Q_pp+Q_sc)*Vao/2

<IPython.core.display.Latex object>

## 2 - Deteminação da Força de Protensão:

### 2.1 - Limites de Tensão:

In [7]:
%%render

sigma_ct0 = -1.2*0.03*(f_cj*10)**(2/3)
sigma_cc0 = 0.7*f_cj
sigma_ctF = -1.2*0.03*(f_ck*10)**(2/3)
sigma_ccF = 0.5*f_ck

r_0 = 0.95
r_F = 0.80

<IPython.core.display.Latex object>

### 2.2 - Determinação da Força de Protensão:

In [8]:
%%render short 2

Fator = (1/Area - e_p/W_s)
##se Fator > 0, a força de protensão deverá ser adotada como: P > P1,P3 e P < P2,P4
##se Fator < 0, a força de protensão deverá ser adotada como: P < P1,P2 e P > P3,P4

P_1 = (sigma_cc0+M_0/W_i)/(r_0*(1/Area + e_p/W_i)) 
P_2 = (sigma_ct0-M_0/W_s)/(r_0*(1/Area - e_p/W_s)) 
P_3 = (sigma_ctF+M_F/W_i)/(r_F*(1/Area + e_p/W_i))
P_4 = (sigma_ccF-M_F/W_s)/(r_F*(1/Area - e_p/W_s))


<IPython.core.display.Latex object>

## 2.3 - Escolha da Protensão:

In [9]:
%%render all
# Força de Protensão Adotada:

P = 2634
phi_1 = 1.52
A_p1 = 1.43

## Número de cordoalhas para adotar:
n = P/(A_p1*sigma_p0)

<IPython.core.display.Latex object>

## 2.4 - Protensão no Apoio:

### 2.4.1 - Comprimento de Ancoragem Básico:

In [59]:
%%render
f_pyd = 0.9* f_ptk / 1.15

eta_p1 = 1.2

# Valores de Eta_p1:
#     
#     1.0 para fios lisos
#     1.2 para cordoalhas de 3 e 7 fios
#     1.5 para fios dentados

eta_p2 = 1.0

# Valores de Eta_p2:
#     
#     1.0 para situação de boa aderência
#     1.2 para situação de má aderência

f_ctd = (0.7*0.03*(f_ck*10)**(2/3))/1.4
f_bpd = eta_p1 * eta_p2 * f_ctd

l_bp = (7*phi_1)/36 * f_pyd/f_bpd

<IPython.core.display.Latex object>

### 2.4.2 - Cálculo do comprimento de Transferência:

In [64]:
%%render

## para fios lisos ou dentados:
l_bpt1 = 0.7 * l_bp * sigma_p0/f_pyd

## para cordoalhas de 3 e 7 fios:
l_bpt2 = 0.5 * l_bp * sigma_p0/f_pyd

## para transferência não gradual, os valores devem ser multiplicados por 1.25
## valor adotado:

l_bpt = 1.25 * l_bpt2


<IPython.core.display.Latex object>

## 2.5 - Escalonamento da Protensão:

In [41]:
%%render

# Vetor com o número de cordoalhas em cada ponto de escalonamento:

qntCordoalhas = ( 5 , 10 , 12 )

# Vetor com a excentricidade da protensão em cada ponto de escalonamento:

e_p_i = [52,52,52]

<IPython.core.display.Latex object>

In [37]:
Areas_p = []

for i in range(0,len(qntCordoalhas)):
    Areas_p.append(vetorEscalonamento[i] * A_p1)

In [42]:
%%render
Areas_p

<IPython.core.display.Latex object>